First, we want to load packages and create an empty data frame to be filled. Additionally, define a starter key to collect more keys

In [1]:
import requests
import json
from bs4 import BeautifulSoup
import urllib
import pandas as pd
import time 
import datetime
final = pd.DataFrame(columns = ['caption','comments','filters','hashtags','image','photo_id','likes','time','user', 'scrape_time', 'key', 'tag']) 
#chance this key will have to be changed over time - atm it's just to start collecting other keys
key = 'AQDt5LYE7lbm7Rb2P-ox-rkpSdXXT19HsvYz4Ncf_k2qcjJ3S2crfI9m-jipFD0SOtWBTAIEKkU1AozNIhN52-wfgMbUTr1biNafa9RGWnQpF6izIF8GIiNf1Dvdl_IbnR4'
counter = 0

Only run the code below if you're looking to get more keys. The number of keys you get will depend on the range

In [ ]:
#collect keys
start = time.time()
result = []  
for i in range(0,1000):
    d = {}
    response = requests.post("http://www.instagim.com/ajax/",
        data='get_photos=https%3A%2F%2Fapi.instagram.com%2Fv1%2Ftags%2Fnature%2Fmedia%2Frecent%3Fmax_tag_id%' + key + '%count%3D15%26access_token%3D&next_url=https%3A%2F%2Fapi.instagram.com%2Fv1%2Ftags%2Fnature%2Fmedia%2Frecent%3Fmax_tag_id%3D%7Bnext_key%7D%26count%3D15%26access_token%3D',
        headers={
            "Accept": "application/json, text/javascript, */*; q=0.01",
            "Accept-Encoding": "gzip, deflate",
            "Accept-Language": "en-US,en;q=0.8,zh-CN;q=0.6,zh;q=0.4",
            "Connection": "keep-alive",
            "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
            "Origin": "http://www.instagim.com",
            "Referer": "http://www.instagim.com/tag/awesome",
            "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36",
            "X-Requested-With": "XMLHttpRequest"
        },
        cookies={
            "_ga": "GA1.2.821769385.1489495570",
            "_gat": "1"
        },
    )
    data = json.loads(response.text)
    d['key'] = data['key'].encode('utf-8')
    d['post_time'] = str(time.strftime("%c"))
    d['tag'] = 'awesome'
    result.append(d)
    time.sleep(1)
    
print 'Time elapsed: ', time.time() - start



Simply writing the keys to a csv file while the photos "mature" like fine wine.

In [3]:
keys_df = pd.DataFrame(result)
print keys_df.head()
#keys_df.to_csv('keys'+ str(time.strftime("%c") + '.csv'))


                                                 key  \
0  AQCru2dacDsmai5xeI3oYLLSQeBJzcwCVDjoYAfWLXNPFg...   
1  AQBRn68jmn9Nuh-Fye5RGRBaZSfVEeo0s4ITCaXl8VokjH...   

                  post_time      tag  
0  Mon Mar 20 09:09:12 2017  awesome  
1  Mon Mar 20 09:09:14 2017  awesome  


Now, we want to use yesterdays keys to actually get the data we want.


In [44]:
key = pd.read_csv('../data/keysThuMar1621_50_502017.csv')
key = key.drop('Unnamed: 0', axis = 1)
key = key.iloc[1:3,:] # for tests
key


,key,post_time
1,AQDRTq-5ApHwFi46pm-JqK-kV15oqfHetPM0JQxNfI1_oC...,Thu Mar 16 21:12:26 2017
2,AQBFhfQpX01OEMbexcfDu1TDZHw8u8rca7PGaYEZEibrjC...,Thu Mar 16 21:12:28 2017


Extract the photos from each key

In [45]:


for index,row in key.iterrows():
    
    response = requests.post("http://www.instagim.com/ajax/",
        data='get_photos=https%3A%2F%2Fapi.instagram.com%2Fv1%2Ftags%2Fnature%2Fmedia%2Frecent%3Fmax_tag_id%3D' + row['key'] + '%26count%3D15%26access_token%3D&next_url=https%3A%2F%2Fapi.instagram.com%2Fv1%2Ftags%2Fnature%2Fmedia%2Frecent%3Fmax_tag_id%3D%7Bnext_key%7D%26count%3D15%26access_token%3D',
        headers={
            "Accept": "application/json, text/javascript, */*; q=0.01",
            "Accept-Encoding": "gzip, deflate",
            "Accept-Language": "en-US,en;q=0.8,zh-CN;q=0.6,zh;q=0.4",
            "Connection": "keep-alive",
            "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
            "Origin": "http://www.instagim.com",
            "Referer": "http://www.instagim.com/tag/nature",
            "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36",
            "X-Requested-With": "XMLHttpRequest"
        },
        cookies={
            "_ga": "GA1.2.821769385.1489495570",
            "_gat": "1"
        },
    )

    data = json.loads(response.text)
    text = BeautifulSoup(data['html'], 'html.parser')
    events = text.find_all('div', {'class': "box-photo"})


    result = [] 
    for event in events:
        counter += 1
        d = {} # each event is its own dictionary so initialize it inside the loop
        try: #use try statement so code doesn't break if names are wrong
                 #remember to change this to a counter
                index = counter
                im = [x['src'] for x in event.find('div', {'class': 'photo'}).a][0]
                urllib.urlretrieve(im, str(counter) + ".jpg")
                time_post = event.find('div', {'class': "upload-time"}).get_text()
                user = event.find('div', {'class': "username"}).a.get_text()
                capt = event.find('div', {'class': "photo-description"}).get_text()
                hashtags = event.find('div', {'class': "photo-tags"}).get_text()
                filters = event.find('div', {'class': "filter-text"}).get_text()
                likes = event.find('span', {'class': "icon-thumbs-up-alt"}).get_text()
                comments = event.find('span', {'class': "icon-chat"}).get_text()
        except: 
                index = ""
                im = ""
                time_post = ""
                user = ""
                capt = ""
                hashtags = ""
                filters = ""
                likes = ""
                comments = ""
        finally:  
                d['photo_id'] = index
                d['image'] = im
                d['time'] = time_post
                d['user'] = user             
                d['caption'] = capt
                d['hashtags'] = hashtags
                d['filters'] = filters
                d['likes'] = likes
                d['comments'] = comments
                d['scrape_time'] = str(time.strftime("%c"))
                d['post_time'] = row['post_time']
                d['key'] = key
                d['tag'] = 'nature'
                result.append(d) # each event is own dictionary

    final = final.append(pd.DataFrame(result))


Double check that your dataframe has been created

In [46]:
final

,caption,comments,filters,hashtags,image,key,likes,photo_id,post_time,scrape_time,tag,time,user
0,\r\n #photo #photography #p...,1,Filter Normal,#spring #alicante #nature #flower #paradise #...,https://scontent.cdninstagram.com/t51.2885-15/...,...,35,1.0,Thu Mar 16 21:12:26 2017,Fri Mar 17 21:07:58 2017,nature,23 hours ago,@hrnndezphoto
1,\r\n #creativesma #nature #...,2,Filter Clarendon,#followme #followforfollow #nature #newsocial...,https://scontent.cdninstagram.com/t51.2885-15/...,...,11,2.0,Thu Mar 16 21:12:26 2017,Fri Mar 17 21:07:58 2017,nature,23 hours ago,@creativesma
2,\r\n #photography #photogra...,4,Filter Normal,#nature #outside #photography #nofilter #niko...,https://scontent.cdninstagram.com/t51.2885-15/...,...,22,3.0,Thu Mar 16 21:12:26 2017,Fri Mar 17 21:07:59 2017,nature,23 hours ago,@stephaniezamudiophotography
3,\r\n 봉은사 홍매화 with Leica SL...,0,Filter Normal,#감성스냅 #이미지메이킹잉글리시q #일상을담은 #스냅 #영어발음의신 #서울 #카페...,https://scontent.cdninstagram.com/t51.2885-15/...,...,21,4.0,Thu Mar 16 21:12:26 2017,Fri Mar 17 21:08:00 2017,nature,23 hours ago,@leica_sl
4,\r\n Selamat pagi kyai dan ...,0,Filter Normal,#natgeo #mtmanolimit #travelling #transtv #tv...,https://scontent.cdninstagram.com/t51.2885-15/...,...,150,5.0,Thu Mar 16 21:12:26 2017,Fri Mar 17 21:08:00 2017,nature,23 hours ago,@sahabatmarshallsastra_lampung
5,\r\n #emo#scene#goth#black#...,1,Filter Clarendon,#red #trippy #guyswithpercings #male #tatto #...,https://scontent.cdninstagram.com/t51.2885-15/...,...,37,6.0,Thu Mar 16 21:12:26 2017,Fri Mar 17 21:08:00 2017,nature,23 hours ago,@daddy_reaper666
6,\r\n #photography #photowit...,0,Filter Normal,#naturelovers #lovefl #naturephotography #for...,https://scontent.cdninstagram.com/t51.2885-15/...,...,13,7.0,Thu Mar 16 21:12:26 2017,Fri Mar 17 21:08:01 2017,nature,1 day ago,@nb_photosoflife
7,\r\n Another side of the ro...,2,Filter Normal,#seattle #lifeisgreat #washingtonstate #smell...,https://scontent.cdninstagram.com/t51.2885-15/...,...,19,8.0,Thu Mar 16 21:12:26 2017,Fri Mar 17 21:08:01 2017,nature,23 hours ago,@shouts_n_gigglez
8,\r\n The girls are about ha...,0,Filter Clarendon,#weightloss #healthy #swfl #cardio #estero #f...,https://scontent.cdninstagram.com/t51.2885-15/...,...,45,9.0,Thu Mar 16 21:12:26 2017,Fri Mar 17 21:08:01 2017,nature,23 hours ago,@johnathanblevinsfitt
9,\r\n #sunrise #sunset #sun ...,2,Filter Ludwig,#red #pink #clouds #instasky #morning #sunset...,https://scontent.cdninstagram.com/t51.2885-15/...,...,71,10.0,Thu Mar 16 21:12:26 2017,Fri Mar 17 21:08:01 2017,nature,23 hours ago,@schiunyix


And finally write that file to a csv!

In [7]:
final.to_csv('Insta.csv', encoding = 'utf-8')

Below is used to get the follower count

In [61]:
#ex users
users = ['silberstil','realdonaldtrump']
counts = []
for user in users:
    followers = requests.get('https://instaliga.com/' + str(user))
    text = BeautifulSoup(followers.text)
    for thing in text.find_all('div', {'class': "userInfo"}):
        counts.append(thing.find('div').text)
print counts

[u'423', u'3225', u'2691', u'1600', u'6277200', u'6']


In [59]:
pd.DataFrame([4,5,5])

,0
0,4
1,5
2,5


In [12]:
print followers.text

<!DOCTYPE html><html lang="en"><head><link rel="dns-prefetch" href="//scontent.cdninstagram.com"><meta charset="UTF-8" /><meta name="viewport" content="width=device-width, initial-scale=1.0, user-scalable=no" /><meta http-equiv="X-UA-COMPATIBLE" content="IE=edge" /><meta name="yandex-verification" content="7056f6a8f8129c46" /><meta name="keywords" content="@silberstil, Thomas Sabo🎀silver 925, Instaliga, Instagram"><meta name="description" content="@silberstil, Thomas Sabo🎀silver 925, Instaliga, Instagram"><meta name="og:title" content="Thomas Sabo🎀silver 925 (@silberstil)" /><meta name="og:type" content="article" /><meta name="og:url" content="http://instaliga.com/silberstil" /><meta name="og:image" content="https://scontent.cdninstagram.com/t51.2885-19/s150x150/17333261_742759159234394_3180069489051959296_a.jpg" /><meta name="og:description" content="Thomas Sabo🎀silver 925 (@silberstil)" /><title>Thomas Sabo🎀silver 925 (@silberstil) - Instaliga is the best instagram web-viewer</title>